In [1]:
import geopandas as gpd
import json
from shapely.ops import nearest_points
import pandas as pd
import fiona
from shapely.geometry import shape,mapping, Point, Polygon, MultiPolygon
from shapely.ops import cascaded_union

In [2]:
admin2 = gpd.read_file('gadm36_ETH_shp/gadm36_ETH_3.shp')
admin2['Gadm Name 2'] = admin2['NAME_2']
admin2['Gadm Name 1'] = admin2['NAME_1']

In [3]:
admin2.groupby(['Gadm Name 1'])['NAME_2'].count()

Gadm Name 1
Addis Abeba                                     10
Afar                                            32
Amhara                                         129
Benshangul-Gumaz                                20
Dire Dawa                                        2
Gambela Peoples                                 13
Harari People                                    1
Oromia                                         260
Somali                                          53
Southern Nations, Nationalities and Peoples    136
Tigray                                          34
Name: NAME_2, dtype: int64

In [4]:
pts3 = admin2.centroid.unary_union

In [5]:
from shapely.ops import nearest_points
# unary union of the gpd2 geomtries 

def near(point, pts=pts3):
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 2'].get_values()[0]


def near_gadm1(geom, pts=pts3):
    if geom.type == Polygon:
        point = geom.centroid
    else:
        point = geom
     # find the nearest point and return the corresponding Place value
    nearest = admin2.centroid == nearest_points(point, pts)[1]
    return admin2[nearest]['Gadm Name 1'].get_values()[0]

In [5]:
maln = gpd.read_file('malnutrition_eth.geojson' )
pop = gpd.read_file("population_ETH_2018.geojson")
travel = gpd.read_file("travel_time.geojson")

In [6]:
flood = pd.read_csv('flood-model.csv')
flood = gpd.GeoDataFrame(
    flood, geometry=gpd.points_from_xy(flood.longitude, flood.latitude))

In [7]:
chirps = pd.read_csv('CHIRPS-combined.csv')

chirps = gpd.GeoDataFrame(
    chirps, geometry=gpd.points_from_xy(chirps.longitude, chirps.latitude))

In [8]:
pop = pop[pop['Population']>0]

In [9]:
maln['centroid']=maln.geometry.apply(lambda x: x.centroid)
pop['centroid']=pop.geometry.apply(lambda x: x.centroid)
admin2['centroid']=admin2.geometry.apply(lambda x: x.centroid)

In [10]:
pop['Gadm Name 2'] = pop.apply(lambda row: near(row.centroid), axis=1)
maln['Gadm Name 2'] = maln.apply(lambda row: near(row.centroid), axis=1)

# for Flood and CHIRPS, just use geometry, not centroid since these are just points not shapes
flood['Gadm Name 2'] = flood.apply(lambda row: near(row.geometry), axis=1)
chirps['Gadm Name 2'] = chirps.apply(lambda row: near(row.geometry), axis=1)

In [11]:
del(pop['centroid'])
del(maln['centroid'])

In [12]:
pop.to_file("pop_ETH.geojson", driver="GeoJSON")
maln.to_file("maln_ETH.geojson", driver="GeoJSON")
flood.to_file("flood_ETH.geojson", driver="GeoJSON")
chirps.to_file("CHIRPS_combined.geojson", driver="GeoJSON")

### Post processing CHIRPS to handle data outside Ethiopia

In [36]:
c = gpd.read_file("CHIRPS_combined.geojson")

Here we read in the Ethiopia shape, merge it to one large polygon, and detect points which are within Ethiopia.

We have to do this because our prior processing mapped a GADM 2 admin to every point, even those in South Sudan.

In [58]:
polys = []
multipol = fiona.open("gadm36_ETH_shp/gadm36_ETH_1.shp")
for n in multipol:
    polys.append(Polygon(n['geometry']['coordinates'][0]))
    
u = cascaded_union(polys)

In [61]:
c['Ethiopia'] = c.geometry.apply(lambda x: x.within(u))

In [62]:
c.groupby(['Ethiopia']).geometry.count()

Ethiopia
False    349067
True     117668
Name: geometry, dtype: int64

In [64]:
def get_gadm(row): 
    if row.Ethiopia == True:
        return row['Gadm Name 2']
    else:
        return None

In [67]:
c['Gadm ETH'] = c.apply(lambda x: get_gadm(x), axis=1)

In [71]:
c['Gadm Name 2'] = c['Gadm ETH']

In [72]:
c.to_file("CHIRPS_combined.geojson", driver="GeoJSON")

## Processing DSSAT

In [87]:
dssat = pd.read_csv('dssat_combined.csv')

dssat = gpd.GeoDataFrame(
    dssat, geometry=gpd.points_from_xy(dssat.LONGITUDE, dssat.LATITUDE))

In [88]:
dssat.shape

(51843, 15)

In [90]:
dssat['Gadm Name 2'] = dssat.apply(lambda row: near(row.geometry), axis=1)

In [5]:
dssat['Year'] = dssat.HDAT.apply(lambda x: int(str(x)[:4]))

In [91]:
dssat.to_file("dssat.geojson", driver="GeoJSON")

## Processing Atlas Model

In [6]:
consumption = pd.read_csv('consumption.csv')

consumption = gpd.GeoDataFrame(
    consumption, geometry=gpd.points_from_xy(consumption.longitude, consumption.latitude))

In [8]:
consumption.shape

(11314, 5)

In [9]:
consumption['Gadm Name 2'] = consumption.apply(lambda row: near(row.geometry), axis=1)

In [11]:
consumption['Gadm Name 1'] = consumption.apply(lambda row: near_gadm1(row.geometry), axis=1)

In [14]:
consumption.to_csv("consumption-processed.csv")

In [18]:
1299204/100000


12.99204

In [15]:
consumption

,Unnamed: 0,consumption,latitude,longitude,geometry,Gadm Name 2,Gadm Name 1
0,0,55.070782,1299204.0,4161468.0,POINT (4161468.000 1299204.000),Doolo,Somali
1,1,48.673542,1255228.0,3901436.0,POINT (3901436.000 1255228.000),Doolo,Somali
2,2,50.241917,1255228.0,3907172.0,POINT (3907172.000 1255228.000),Doolo,Somali
3,3,48.746292,1255228.0,3912908.0,POINT (3912908.000 1255228.000),Doolo,Somali
4,4,70.044632,1262876.0,3907172.0,POINT (3907172.000 1262876.000),Doolo,Somali
5,5,66.520958,1262876.0,3912908.0,POINT (3912908.000 1262876.000),Doolo,Somali
6,6,46.463074,1293468.0,4161468.0,POINT (4161468.000 1293468.000),Doolo,Somali
7,7,41.187057,1306852.0,4161468.0,POINT (4161468.000 1306852.000),Doolo,Somali
8,8,15.304705,1245668.0,3910996.0,POINT (3910996.000 1245668.000),Doolo,Somali
9,9,62.435699,1262876.0,3899524.0,POINT (3899524.000 1262876.000),Doolo,Somali


## Admin Level 1
Add back in admin level 1 (State) to all files

In [30]:
admin1 = admin2[['NAME_2','NAME_1']].drop_duplicates().set_index("NAME_2")

In [55]:
def find_admin1(admin2):
    if admin2==None:
        return "Null"
    else:
        return str(admin1.loc[admin2].NAME_1)

In [56]:
find_admin1("North Shewa")

'NAME_2\nNorth Shewa    Amhara\nNorth Shewa    Oromia\nName: NAME_1, dtype: object'

In [8]:
pop = gpd.read_file("pop_ETH.geojson")
maln = gpd.read_file("maln_ETH.geojson")
flood = gpd.read_file("flood_ETH.geojson")
chirps = gpd.read_file("CHIRPS_combined.geojson")
dssat = gpd.read_file("dssat.geojson")

In [51]:
pop['Gadm Name 1'] = pop['Gadm Name 2'].apply(lambda x: find_admin1(x))
maln['Gadm Name 1'] = maln['Gadm Name 2'].apply(lambda x: find_admin1(x))
flood['Gadm Name 1'] = flood['Gadm Name 2'].apply(lambda x: find_admin1(x))
chirps['Gadm Name 1'] = chirps['Gadm Name 2'].apply(lambda x: find_admin1(x))
dssat['Gadm Name 1'] = dssat['Gadm Name 2'].apply(lambda x: find_admin1(x))

Reprocess nearest gadm for North Shewa since it exists in Oromia and Amhara; can't do a simple lookup!

Exclude flood model since it doesn't have points in North Shewa of either Region

In [100]:
dssat.loc[dssat['Gadm Name 2']=='North Shewa', 'Gadm Name 1'] = dssat[dssat['Gadm Name 2']=='North Shewa']\
                                                            .geometry.apply(lambda x: near_gadm1(x))

chirps.loc[chirps['Gadm Name 2']=='North Shewa', 'Gadm Name 1'] = chirps[chirps['Gadm Name 2']=='North Shewa']\
                                                            .geometry.apply(lambda x: near_gadm1(x))

pop.loc[pop['Gadm Name 2']=='North Shewa', 'Gadm Name 1'] = pop[pop['Gadm Name 2']=='North Shewa']\
                                                            .geometry.apply(lambda x: near_gadm1(x))

maln.loc[maln['Gadm Name 2']=='North Shewa', 'Gadm Name 1'] = maln[maln['Gadm Name 2']=='North Shewa']\
                                                            .geometry.apply(lambda x: near_gadm1(x))

In [106]:
pop.to_file("pop_ETH.geojson", driver="GeoJSON")
maln.to_file("maln_ETH.geojson", driver="GeoJSON")
flood.to_file("flood_ETH.geojson", driver="GeoJSON")
chirps.to_file("CHIRPS_combined.geojson", driver="GeoJSON")
dssat.to_file("dssat.geojson", driver="GeoJSON")